In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input, BatchNormalization, ZeroPadding2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras import optimizers
import numpy as np
from sklearn.metrics import accuracy_score

In [ ]:
DROPOUT = 0.5
N_CATEGORY = 3
model_input = Input(shape = (3, 227, 227))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), sstrides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)

In [ ]:
WEIGHT_DECAY = 0.0005
MOMENTUM = 0.9
BATCH_SIZE = 128
LEARNING_RATE = 0.01
DROPOUT = 0.5
ALPHA = 1e-4
BETA = 0.75
n = 5
k = 2

loss_metric = "binary_crossentropy" if N_CATEGORY == 2 else "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.SGD(lr=LEARNING_RATE, momentum=MOMENTUM, decay = WEIGHT_DECAY))

In [ ]:
model.load_weights('Workshop-Paper/Alexnet-Gaussian/vanila-raw-085-0.560')

In [ ]:
test_data_gen = ImageDataGenerator(rescale=1.0/100)
test_data_stream = test_data_gen.flow_from_directory(
                    'Data/Heatmaps/Train-Simple-Float-Gauss/ValidateFloatGaussian/',
                    target_size=(227, 227),
                    class_mode='categorical',
                    shuffle=False,
                    batch_size=1
                    )

In [ ]:
predictions = model.predict_generator(test_data_stream, 1979, verbose=1)
actual_classes = test_data_stream.classes
pred_classes = np.argmax(predictions, axis=1)


In [ ]:
print ('Alexnet Gaussian Validation Accuracy Score:', accuracy_score(pred_classes, actual_classes))

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1.0/100)
train_data_stream = train_data_gen.flow_from_directory(
                    'Data/Heatmaps/Train-Simple-Float-Gauss/TrainFloatGaussian/',
                    target_size=(227, 227),
                    class_mode='categorical',
                    shuffle=False,
                    batch_size=1
                    )

In [ ]:
predictions = model.predict_generator(train_data_stream, 3157, verbose=1)
actual_classes = train_data_stream.classes
pred_classes = np.argmax(predictions, axis=1)
print ('Alexnet Gaussian Training Accuracy Score:', accuracy_score(pred_classes, actual_classes))